In [3]:
#importing library
from bs4 import BeautifulSoup
import requests
import time
import re
from datetime import datetime
import html5lib



In [16]:
web_24h_com_vn = {
        "home_page":"https://www.24h.com.vn/",
        "urls":{
            "tech":
            {
             "url":"https://www.24h.com.vn/cong-nghe-thong-tin-c55.html",
             "cate_id":57,
             "sub-category":{
                0:{"name":"Game",
                 "url":"https://www.24h.com.vn/game-c69.html"},
                1:{"name":"Phần mềm",
                 "url":"https://www.24h.com.vn/phan-mem-ngoai-c302.html"},
                2:{"name":"Khoa học",
                 "url":"https://www.24h.com.vn/khoa-hoc-c782.html"},
                3:{"name":"Mạng xã hội",
                 "url":"https://www.24h.com.vn/mang-xa-hoi-c889.html"},
                4:{"name":"Thủ thuật - Tiện ích"
                 ,"url":"https://www.24h.com.vn/thu-thuat-tien-ich-c68.html"},
                5:{"name":"Sợ Virus",
                 "url":"https://www.24h.com.vn/tim-hieu-virus-c57.html"},
                6:{"name":"Máy in/phụ kiện",
                 "url":"https://www.24h.com.vn/may-in/phu-kien-c291.html"},
                7:{"name":"Khám phá công nghệ",
                 "url":"https://www.24h.com.vn/kham-pha-cong-nghe-c675.html"}
             }
            }
            ,
            "youths":
            {
            "url":"https://www.24h.com.vn/ban-tre-cuoc-song-c64.html",
            "cate_id":60,
             "sub-category":{
                0:{"name":"Chuyện công sở","url":"https://www.24h.com.vn/chuyen-cong-so-c180.html"},
                1:{"name":"Tình yêu - Giới Tính","url":"https://www.24h.com.vn/tinh-yeu-gioi-tinh-c306.html"},
                2:{"name":"Ngoại tình","url":"https://www.24h.com.vn/ngoai-tinh-c435.html"},
                3:{"name":"Giới trẻ","url":"https://www.24h.com.vn/gioi-tre-c434.html"},
                4:{"name":"Hotgirl - Hotboy","url":"https://www.24h.com.vn/hotgirl-hot-boy-c64e3398.html"},
                5:{"name":"Nhịp sống trẻ","url":"https://www.24h.com.vn/nhip-song-tre-c685.html"}
             }
            }
            ,
            "showbiz":
            {
            "cate_id":59,
            "url":"https://www.24h.com.vn/doi-song-showbiz-c729.html",
             "sub-category":{
                0:{"name":"Sao Việt","url":"https://www.24h.com.vn/sao-viet-c757.html"},
                1:{"name":"24h gặp gỡ","url":"https://www.24h.com.vn/gap-go-24h-c729e6820.html"},
                2:{"name":"Talk với sao","url":"https://www.24h.com.vn/doi-thoai-cung-sao-c730.html"},
                3:{"name":"Sao châu Á","url":"https://www.24h.com.vn/sao-chau-a-c759.html"},
            }
            },
            "cars":
            {
            "cate_id":58,
            "url":"https://www.24h.com.vn/o-to-c747.html",
             "sub-category":{
                0:{"name":"Tin tức ô tô","url":"https://www.24h.com.vn/tin-tuc-o-to-c332.html"},
                1:{"name":"Bảng giá xe ô tô","url":"https://www.24h.com.vn/bang-gia-xe-o-to-c807.html"},
                2:{"name":"Tư vấn","url":"https://www.24h.com.vn/tu-van-c240.html"},
                3:{"name":"Ngắm xe","url":"https://www.24h.com.vn/anh-nguoi-dep-va-xe-c199.html"},
                4:{"name":"Đánh giá xe","url":"https://www.24h.com.vn/so-sanh-xe-c805.html"},
            }
            }
        }
    }

In [147]:
def get_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content,  features='html')

    #remove the last 4 element
    article = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info')
    
    children = article.find_all(recursive=False)
    for i in children[-4:]:
        i.decompose()
    caption_text_list =article.find_all('p',class_='img_chu_thich_0407')
    #find a,span and remove tag with text
    tags_to_remove = article.find_all(['a', 'span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
        # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip()
    # set attribut for img
    b = article.find_all('img')
    for i in b:
        if '.svg' in i['src']:
            i.decompose()
    for i in article.find_all('img'):
        if 'data-original' in i.attrs.keys():
            i['src'] = i.get('data-original')
        #i.attrs = ['class', 'alt', 'src', 'data-original']
        del i['onclick']
        del i['style']
        del i['class']
        
        
        #print(n_img)

    img_list = article.find_all('img')
    n_img = len(img_list)
    #print(len(caption_text_list))
    for i in range(0,n_img):
        caption_tag = soup.new_tag('caption')
        img_list[i].insert_before(caption_tag)
        caption_tag.append(img_list[i])
        caption_tag['width'] = 800
        caption_tag['align'] ="aligncenter"
        try:
            caption_tag.append(caption_text_list[i].string)
        except IndexError as e:
            print(e)
            #create caption
            
        #print(i.attrs.keys())"""
    #remove js  and css
    h2 = soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find('h2')
    #soup2 = BeautifulSoup("", 'html.parser')
    #soup2.append(h2)
    for script_or_style in article(['script', 'style']):
        script_or_style.decompose()
    for i in article.find_all(recursive = True):
        try:
            del i['onclick']
            del i['id']
            del i['class']
            del i['style']
        except AttributeError:
            continue
        try:
            print(i)
            
        except TypeError:
            continue
    for i in article.find_all('img'):
        i['class'] = "aligncenter"
        i['width'] = 800
        i['height'] = 400
    for item in article.find_all('div'):
        if item.string =="":
            item.decompose()
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: 24h.com.vn"  # Set the content of <i> tag

# Append the <i> tag as the last child of the <article> tag
    article.append(source_tag)
    return str(article)
#convert time from post to the format of output
def convert_string(input_str):
    # Extract the date part using regular expression
    pattern = r'\d{2}/\d{2}/\d{4}'
    date_part = re.search(pattern, input_str).group()
    match = re.search(pattern, input_str)
    if match:
        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_part, '%d/%m/%Y')
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    else:
        return ""
#Convert time from post to datetime format
def convert_time_string(posted_date):
    pattern = r'\d{2}/\d{2}/\d{4}'
    match = re.search(pattern, posted_date)

    if match:
        date_string = match.group()

        # Convert to datetime object
        datetime_obj = datetime.strptime(date_string, "%d/%m/%Y")
        return datetime_obj
    else:
        return ''
def get_post(url):
    try:
        response = requests.get(url)
        time.sleep(3)
        soup = BeautifulSoup(response.content, 'html5lib')
        content = get_content(url)
        post_time = soup.find('time').text.strip()
        published_date = convert_string(post_time)
        title = soup.find('h1').text.strip()
        #h2 = soup.find('h2').text.strip()
        #images_src = [i.attrs['src'] for i in soup.find('article', class_= 'cate-24h-foot-arti-deta-info').find_all('img')[:-1] if 'svg' not in i.attrs['src']]
        #images_src = [img['data-original'] if 'https://image-us.24h.com.vn' not in img['src'] else img['src'] if 'svg' not in img['src'] else '' for img in soup.find('article', class_ = 'cate-24h-foot-arti-deta-info').find_all('img')[:-1]]
        #text_list = [ child.text for child in soup.find('article', class_= 'cate-24h-foot-arti-deta-info').find_all('p')[:-3] if re.sub(r'\n+', '', child.text) != ""]
        #text_list = [h2] + text_list
        return content,title,published_date
    except AttributeError as e:
        print(e)
        content = ''
        title = ''
        published_date= ''
        return content,title,published_date

def filter_list(urls):
    filtered_urls = []
    crawl_time = datetime.fromtimestamp(time.time() - 3*24*3600)
    for i in urls:
        response = requests.get(i)
        soup = BeautifulSoup(response.content, 'html5lib')
        try:
            date_posted = soup.find('time').text.strip()
            date_posted_norm = convert_time_string(date_posted)
            if ( (date_posted_norm.day == crawl_time.day) and (date_posted_norm.month == crawl_time.month) and (date_posted_norm.year == crawl_time.year) ):
                filtered_urls.append(i)
                print(i)
        except AttributeError as e:
            print(e)
            break
    return filtered_urls
def get_list_url(cat_url):
    urls = []
    response = requests.get(cat_url)
    time.sleep(3)
    soup = BeautifulSoup(response.content,'html5lib')
    news_block  = soup.find('section', id = 'tin_bai_noi_bat_khac')
    try:
        url_list = news_block.find_all('a')
        print(len(url_list))
        for url in url_list:
            if 'https://www.24h.com.vn/' in url['href']:
                if url['href'] not in urls:
                    urls.append(url['href'])
    except AttributeError as e:
        print(e)
    return urls
def add_list(web_24h_com_vn):
    for i in list(web_24h_com_vn['urls'].keys()):
        for j in list(web_24h_com_vn['urls'][i]['sub-category'].keys()):
            
            urls = get_list_url(web_24h_com_vn['urls'][i]['sub-category'][j]['url'])
            print(i,j,web_24h_com_vn['urls'][i]['sub-category'][j]['url'])
            web_24h_com_vn['urls'][i]['sub-category'][j]['url_list'] = filter_list(urls)

def add_post(web_24h_com_vn):
    for i in list(web_24h_com_vn['urls'].keys()):
        for j in list(web_24h_com_vn['urls'][i]['sub-category'].keys()):
            web_24h_com_vn['urls'][i]['sub-category'][j]['content'] = {}
            list_key = [v for v in range(0,len(web_24h_com_vn['urls'][i]['sub-category'][j]['url_list']))]
            for u in list_key:
                web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u] = {}
                if u != "":
                    web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['text'] ,web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['title'],web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['published_date'] = get_post(web_24h_com_vn['urls'][i]['sub-category'][j]['url_list'][u])
                    print(i,j,web_24h_com_vn['urls'][i]['cate_id'],web_24h_com_vn['urls'][i]['sub-category'][j]['name'],web_24h_com_vn['urls'][i]['sub-category'][j]['name'],web_24h_com_vn['urls'][i]['sub-category'][j]['content'][u]['title'],web_24h_com_vn['urls'][i]['sub-category'][j]['url_list'][u])
                




In [149]:
def get_news():
    web_24h_com_vn = {
        "home_page":"https://www.24h.com.vn/",
        "urls":{
            "tech":
            {
             "url":"https://www.24h.com.vn/cong-nghe-thong-tin-c55.html",
             "cate_id":57,
             "sub-category":{
                0:{"name":"Game",
                 "url":"https://www.24h.com.vn/game-c69.html"},
                1:{"name":"Phần mềm",
                 "url":"https://www.24h.com.vn/phan-mem-ngoai-c302.html"},
                2:{"name":"Khoa học",
                 "url":"https://www.24h.com.vn/khoa-hoc-c782.html"},
                3:{"name":"Mạng xã hội",
                 "url":"https://www.24h.com.vn/mang-xa-hoi-c889.html"},
                4:{"name":"Thủ thuật - Tiện ích"
                 ,"url":"https://www.24h.com.vn/thu-thuat-tien-ich-c68.html"},
                5:{"name":"Sợ Virus",
                 "url":"https://www.24h.com.vn/tim-hieu-virus-c57.html"},
                6:{"name":"Máy in/phụ kiện",
                 "url":"https://www.24h.com.vn/may-in/phu-kien-c291.html"},
                7:{"name":"Khám phá công nghệ",
                 "url":"https://www.24h.com.vn/kham-pha-cong-nghe-c675.html"}
             }
            }
            ,
            "youths":
            {
            "url":"https://www.24h.com.vn/ban-tre-cuoc-song-c64.html",
            "cate_id":60,
             "sub-category":{
                0:{"name":"Chuyện công sở","url":"https://www.24h.com.vn/chuyen-cong-so-c180.html"},
                1:{"name":"Tình yêu - Giới Tính","url":"https://www.24h.com.vn/tinh-yeu-gioi-tinh-c306.html"},
                2:{"name":"Ngoại tình","url":"https://www.24h.com.vn/ngoai-tinh-c435.html"},
                3:{"name":"Giới trẻ","url":"https://www.24h.com.vn/gioi-tre-c434.html"},
                4:{"name":"Hotgirl - Hotboy","url":"https://www.24h.com.vn/hotgirl-hot-boy-c64e3398.html"},
                5:{"name":"Nhịp sống trẻ","url":"https://www.24h.com.vn/nhip-song-tre-c685.html"}
             }
            }
            ,
            "showbiz":
            {
            "cate_id":59,
            "url":"https://www.24h.com.vn/doi-song-showbiz-c729.html",
             "sub-category":{
                0:{"name":"Sao Việt","url":"https://www.24h.com.vn/sao-viet-c757.html"},
                1:{"name":"24h gặp gỡ","url":"https://www.24h.com.vn/gap-go-24h-c729e6820.html"},
                2:{"name":"Talk với sao","url":"https://www.24h.com.vn/doi-thoai-cung-sao-c730.html"},
                3:{"name":"Sao châu Á","url":"https://www.24h.com.vn/sao-chau-a-c759.html"},
            }
            },
            "cars":
            {
            "cate_id":58,
            "url":"https://www.24h.com.vn/o-to-c747.html",
             "sub-category":{
                0:{"name":"Tin tức ô tô","url":"https://www.24h.com.vn/tin-tuc-o-to-c332.html"},
                1:{"name":"Bảng giá xe ô tô","url":"https://www.24h.com.vn/bang-gia-xe-o-to-c807.html"},
                2:{"name":"Tư vấn","url":"https://www.24h.com.vn/tu-van-c240.html"},
                3:{"name":"Ngắm xe","url":"https://www.24h.com.vn/anh-nguoi-dep-va-xe-c199.html"},
                4:{"name":"Đánh giá xe","url":"https://www.24h.com.vn/so-sanh-xe-c805.html"},
            }
            }
        }
    }
#
    add_list(web_24h_com_vn)
    add_post(web_24h_com_vn)
    return web_24h_com_vn
def send_post_to_5goals(title,content,category_id,published_date):
    # URL of the API endpoint (this is a placeholder and needs to be replaced with the actual URL)
    url = "https://api2023.5goal.com/wp-json/custom/createPost"
    
    # Data to be sent in the POST request
    data = {
        "title": title,
        "content": content,
        "category_id": category_id,
        "token": '5goalvodichcmnl',  # Replace with your actual access token
        "published_date": published_date
          # Replace with the actual category ID as required
    }
    
    # Sending the POST request
    response = requests.post(url, data=data)
    
    # Checking the response
    if response.status_code == 200:
        print("The post was successfully created.")
        print("Response:", response.text)  # Prints the response text from the server
    else:
        print(f"Failed to create the post. Status code: {response.status_code}")
def main():
    web_24h_com_vn = get_news()
    for i in list(web_24h_com_vn['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(web_24h_com_vn['urls'][i]['sub-category']):
            url_list =  web_24h_com_vn['urls'][i]['sub-category'][j]['url_list']
            for t in range(0,len(url_list)):
                text = web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['text']
                title = web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = web_24h_com_vn['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = web_24h_com_vn['urls'][i]['cate_id']
                send_post_to_5goals(title,text,cate_id,published_date)
                time.sleep(5)
    return web_24h_com_vn

<h2>
<strong>
                    Thoạt nghe câu hỏi này có vẻ rất vô lý nhưng bạn cần phải suy nghĩ và mở rộng tư duy một chút mới giải đáp được.                </strong>
</h2>
<strong>
                    Thoạt nghe câu hỏi này có vẻ rất vô lý nhưng bạn cần phải suy nghĩ và mở rộng tư duy một chút mới giải đáp được.                </strong>
<p>Người ta thường nói rằng, tìm việc là một cuộc cạnh tranh gay gắt hơn cả thi cử. Bằng cấp cao, thành tích tốt chưa chắc đã có thể vượt qua vòng phỏng vấn. Cuộc chiến giữa những ứng viên bây giờ không chỉ còn là IQ mà phải có cả EQ. Làm thế nào để có thể nổi bật giữa rất nhiều người trong buổi phỏng vấn là điều rất quan trọng.</p>
<p>Vương Quân (Trung Quốc) tốt nghiệp từ một trường đại học top đầu, cô có ngoại hình nổi trội và đầu óc rất nhạy bén. Sau khi tốt nghiệp, cô nhận được lời mời phỏng vấn của một công ty lớn. Cô đã bỏ rất nhiều công sức cho buổi phỏng vấn này nhưng có một câu hỏi khiến cô cảm thấy rất kỳ lạ.</p>
<p align="center"></p>
<

'<article class="cate-24h-foot-arti-deta-info">\n<h2>\n<strong>\r\n                    Thoạt nghe câu hỏi này có vẻ rất vô lý nhưng bạn cần phải suy nghĩ và mở rộng tư duy một chút mới giải đáp được.                </strong>\n</h2>\n <p>Người ta thường nói rằng, tìm việc là một cuộc cạnh tranh gay gắt hơn cả thi cử. Bằng cấp cao, thành tích tốt chưa chắc đã có thể vượt qua vòng phỏng vấn. Cuộc chiến giữa những ứng viên bây giờ không chỉ còn là IQ mà phải có cả EQ. Làm thế nào để có thể nổi bật giữa rất nhiều người trong buổi phỏng vấn là điều rất quan trọng.</p>\n<p>Vương Quân (Trung Quốc) tốt nghiệp từ một trường đại học top đầu, cô có ngoại hình nổi trội và đầu óc rất nhạy bén. Sau khi tốt nghiệp, cô nhận được lời mời phỏng vấn của một công ty lớn. Cô đã bỏ rất nhiều công sức cho buổi phỏng vấn này nhưng có một câu hỏi khiến cô cảm thấy rất kỳ lạ.</p>\n<p align="center"></p><div>\n<div>\n<caption align="aligncenter" width="800"><img alt="Cho 8 hạt đậu vào 3 cái bát sao cho mỗi bên là

In [143]:
#content, title, date = get_post('https://www.24h.com.vn/ban-tre-cuoc-song/do-khoc-do-cuoi-chuyen-sep-doi-nhan-vien-cu-tra-tien-tra-sua-c64a1500980.html')


In [150]:
web_24h_com_vn = main()
web_24h_com_vn

23
tech 0 https://www.24h.com.vn/game-c69.html
23
tech 1 https://www.24h.com.vn/phan-mem-ngoai-c302.html
23
tech 2 https://www.24h.com.vn/khoa-hoc-c782.html
23
tech 3 https://www.24h.com.vn/mang-xa-hoi-c889.html
23
tech 4 https://www.24h.com.vn/thu-thuat-tien-ich-c68.html
23
tech 5 https://www.24h.com.vn/tim-hieu-virus-c57.html
23
tech 6 https://www.24h.com.vn/may-in/phu-kien-c291.html
23
tech 7 https://www.24h.com.vn/kham-pha-cong-nghe-c675.html
https://www.24h.com.vn/kham-pha-cong-nghe/cong-ty-tnhh-ky-thuat-tu-dong-hung-phat-don-vi-noi-tieng-trong-linh-vuc-cong-nghe-tu-dong-c675a1520216.html
https://www.24h.com.vn/kham-pha-cong-nghe/vinaphone-la-mang-di-dong-nhanh-nhat-viet-nam-nam-2023-c675a1520094.html
28
youths 0 https://www.24h.com.vn/chuyen-cong-so-c180.html
'NoneType' object has no attribute 'text'
29
youths 1 https://www.24h.com.vn/tinh-yeu-gioi-tinh-c306.html
'NoneType' object has no attribute 'text'
29
youths 2 https://www.24h.com.vn/ngoai-tinh-c435.html
29
youths 3 https://

{'home_page': 'https://www.24h.com.vn/',
 'urls': {'tech': {'url': 'https://www.24h.com.vn/cong-nghe-thong-tin-c55.html',
   'cate_id': 57,
   'sub-category': {0: {'name': 'Game',
     'url': 'https://www.24h.com.vn/game-c69.html',
     'url_list': [],
     'content': {}},
    1: {'name': 'Phần mềm',
     'url': 'https://www.24h.com.vn/phan-mem-ngoai-c302.html',
     'url_list': [],
     'content': {}},
    2: {'name': 'Khoa học',
     'url': 'https://www.24h.com.vn/khoa-hoc-c782.html',
     'url_list': [],
     'content': {}},
    3: {'name': 'Mạng xã hội',
     'url': 'https://www.24h.com.vn/mang-xa-hoi-c889.html',
     'url_list': [],
     'content': {}},
    4: {'name': 'Thủ thuật - Tiện ích',
     'url': 'https://www.24h.com.vn/thu-thuat-tien-ich-c68.html',
     'url_list': [],
     'content': {}},
    5: {'name': 'Sợ Virus',
     'url': 'https://www.24h.com.vn/tim-hieu-virus-c57.html',
     'url_list': [],
     'content': {}},
    6: {'name': 'Máy in/phụ kiện',
     'url': 'https

In [126]:
get_content('https://www.24h.com.vn/ban-tre-cuoc-song/cho-8-hat-dau-vao-3-cai-bat-sao-cho-moi-ben-la-so-le-cau-tra-loi-giup-co-gai-dau-phong-van-c64a1512676.html')

{'home_page': 'https://www.24h.com.vn/',
 'urls': {'tech': {'url': 'https://www.24h.com.vn/cong-nghe-thong-tin-c55.html',
   'cate_id': 57,
   'sub-category': {0: {'name': 'Game',
     'url': 'https://www.24h.com.vn/game-c69.html',
     'url_list': [],
     'content': {}},
    1: {'name': 'Phần mềm',
     'url': 'https://www.24h.com.vn/phan-mem-ngoai-c302.html',
     'url_list': ['https://www.24h.com.vn/cong-nghe-thong-tin/tri-tue-nhan-tao-lan-dau-duoc-ung-dung-vao-phong-luyen-thi-ao-nay-c55a1520038.html'],
     'content': {0: {'text': '<article class="cate-24h-foot-arti-deta-info">\n<h2 class="cate-24h-foot-arti-deta-sum ctTp tuht_show">\n<strong>\r\n                    AI của phòng luyện thi ảo này sẽ chỉ ra các lỗi sai từ vựng, ngữ pháp, lỗi diễn đạt,... cho người học TOEIC.                </strong>\n</h2>\n <p>Prep vừa ra mắt phòng luyện thi ảo TOEIC ứng dụng trí tuệ nhân tạo (AI). Đơn vị này khẳng định, họ là cái tên\xa0đầu tiên và duy nhất trên thị trường, tạo môi trường học tập 

In [ ]:
for i in list(web_24h_com_vn2['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
    for j in list(web_24h_com_vn2['urls'][i]['sub-category']):
        url_list =  web_24h_com_vn2['urls'][i]['sub-category'][j]['url_list']
        for t in range(0,len(url_list)):
            text = web_24h_com_vn2['urls'][i]['sub-category'][j]['content'][t]['text']
            title = web_24h_com_vn2['urls'][i]['sub-category'][j]['content'][t]['title']
            published_date = web_24h_com_vn2['urls'][i]['sub-category'][j]['content'][t]['published_date']
            cate_id = web_24h_com_vn2['urls'][i]['cate_id']
            print(title,published_date,cate_id,text)

In [120]:
def get_list_url(cat_url):
    urls = []
    response = requests.get(cat_url)
    time.sleep(3)
    soup = BeautifulSoup(response.content,'html5lib')
    news_block  = soup.find('section', id = 'tin_bai_noi_bat_khac')
    try:
        url_list = news_block.find_all('a')
        for url in url_list:
            if 'https://www.24h.com.vn/' in url['href']:
                if url['href'] not in urls:
                    urls.append(url['href'])
    except AttributeError as e:
        print(e, hehehehe)
    return urls

In [121]:
get_list_url('https://www.24h.com.vn/chuyen-cong-so-c180.html')

['https://www.24h.com.vn/ban-tre-cuoc-song/cho-8-hat-dau-vao-3-cai-bat-sao-cho-moi-ben-la-so-le-cau-tra-loi-giup-co-gai-dau-phong-van-c64a1512676.html',
 'https://www.24h.com.vn/ban-tre-cuoc-song/khong-du-kpi-co-gai-bi-cac-dong-nghiep-lam-dieu-khien-ai-cung-rung-minh-so-hai-c64a1514624.html',
 'https://www.24h.com.vn/ban-tre-cuoc-song/ngay-dau-di-lam-co-gai-gap-ngay-tinh-huong-dang-xau-ho-c64a1514439.html',
 'https://www.24h.com.vn/ban-tre-cuoc-song/bo-cong-viec-trong-mo-voi-muc-luong-9-ty-nam-o-facebook-vi-khong-chiu-noi-dieu-nay-c64a1514230.html',
 'https://www.24h.com.vn/ban-tre-cuoc-song/that-nghiep-cang-lau-deal-luong-phai-cang-thap-c64a1513277.html',
 'https://www.24h.com.vn/ban-tre-cuoc-song/nha-tuyen-dung-hoi-tren-nguc-cua-ban-la-gi-cai-gi-co-gai-nguong-ngung-tra-loi-va-nhan-cai-ket-bat-ngo-c64a1510902.html',
 'https://www.24h.com.vn/ban-tre-cuoc-song/chuyen-that-nhu-dua-noi-doi-vi-khong-muon-cho-dong-nghiep-vay-tien-co-gai-bi-phat-c64a1506102.html',
 'https://www.24h.com.vn/ba